In [1]:
import re
import pandas as pd
import numpy as np
import os
import requests
import src.utils as u
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from src.exceptions import FilmNotFoundError
from bs4 import BeautifulSoup
from dotenv import load_dotenv

In [2]:
oscars = pd.read_csv("./INPUT/the_oscar_award.csv")
oscars.head()

,year_film,year_ceremony,ceremony,category,name,film,win
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [3]:
(oscars [oscars.isnull()]).count()

year_film        0
year_ceremony    0
ceremony         0
category         0
name             0
film             0
win              0
dtype: int64

# Si el campo película está en blanco es que no hace referencia a una película específica

In [4]:
(oscars.film.value_counts()[oscars.film.value_counts()>1]).head()

                        113
A Star Is Born           25
Titanic                  16
Mutiny on the Bounty     15
Moulin Rouge             15
Name: film, dtype: int64

In [36]:
oscars[oscars.film == " "].head()

,year_film,year_ceremony,ceremony,category,name,film,win
16,1927,1928,1,ENGINEERING EFFECTS,Ralph Hammeras,,False
18,1927,1928,1,ENGINEERING EFFECTS,Nugent Slaughter,,False
31,1927,1928,1,WRITING (Title Writing),Joseph Farnham,,True
32,1927,1928,1,WRITING (Title Writing),George Marion Jr,,False
143,1930,1931,4,SOUND RECORDING,Samuel Goldwyn - United Artists Studio Sound D...,,False


# Para API

In [8]:
u.request_TheMobieDB("Superman", 1978)

True

In [28]:
df = u.get_properties_film(oscars.film, oscars.year_film, oscars, properties=["original_title", "release_date", "genre_ids", "poster_path"])
df

In [82]:
dataframe.original_title.value_counts()[dataframe.original_title.value_counts() > 25]
#dataframe.original_title[dataframe.original_title.isnull()]#Oscar sin pelicula

Film not found    258
Name: original_title, dtype: int64

In [169]:
#dataframe["Genres"] = dataframe.Genres.apply(lambda x: "" if x == None else x)
#dataframe["original_title"] = dataframe.original_title.apply(lambda x: "" if x == None else x)
#dataframe["release_date"] = dataframe.release_date.apply(lambda x: "" if x == None else x)
#dataframe["poster_path"] = dataframe.poster_path.apply(lambda x: "" if x == None else x)

In [170]:
df["Genres"] = u.df.Genres.apply(get_genre)
df

,original_title,release_date,genre_ids,poster_path,Genres
0,The Noose,1928-01-29,[18],[18],[18]
1,The Last Command,1928-01-21,"[18, 36, 10749, 10752]","[18, 36, 10749, 10752]","[18, 36, 10749, 10752]"
2,A Ship Comes In,1928-01-04,[18],[18],[18]
3,7th Heaven,1927-01-01,"[18, 10749]","[18, 10749]","[18, 10749]"
4,Sadie Thompson,1928-01-07,"[18, 10749]","[18, 10749]","[18, 10749]"
...,...,...,...,...,...
10073,First Reformed,2018-05-18,[18],[18],[18]
10074,Green Book,2018-11-16,"[35, 18]","[35, 18]","[35, 18]"
10075,Roma,2018-08-25,[18],[18],[18]
10076,Vice,2018-12-25,"[35, 18, 36]","[35, 18, 36]","[35, 18, 36]"


In [193]:
oscars["Genres"] = dataframe["Genres"]
oscars["Poster Path"] = dataframe["poster_path"]

In [195]:
dataframe

,original_title,release_date,genre_ids,poster_path,Genres
0,The Noose,1928-01-29,[18],[18],[18]
1,The Last Command,1928-01-21,"[18, 36, 10749, 10752]","[18, 36, 10749, 10752]","[18, 36, 10749, 10752]"
2,A Ship Comes In,1928-01-04,[18],[18],[18]
3,7th Heaven,1927-01-01,"[18, 10749]","[18, 10749]","[18, 10749]"
4,Sadie Thompson,1928-01-07,"[18, 10749]","[18, 10749]","[18, 10749]"
...,...,...,...,...,...
10073,First Reformed,2018-05-18,[18],[18],[18]
10074,Green Book,2018-11-16,"[35, 18]","[35, 18]","[35, 18]"
10075,Roma,2018-08-25,[18],[18],[18]
10076,Vice,2018-12-25,"[35, 18, 36]","[35, 18, 36]","[35, 18, 36]"


# Scrapping Con Selenium

In [5]:
u.find_filmaffinity_film("Matrix",1998)

[7.9,
 'Thomas Anderson es un brillante programador de una respetable compañía de software. Pero fuera del trabajo es Neo, un hacker que un día recibe una misteriosa visita... (FILMAFFINITY)']

In [113]:
df2 = u.get_properties_film_filmaffinity(oscars.film[:501], oscars.year_film[:501])
df2

,Rate,Description
0,,Cuando su ex mujer se casa con el rico goberna...
1,7.8,La Última Orden se inspira en una historia sup...
2,,Inmigrantes húngaros viajan en barco a Estados...
3,7.8,Chico (Charles Farrell) es un joven que trabaj...
4,6.8,"A la isla Pago-Pago en el Pacífico, llegan sim..."
...,...,...
239,,
240,7.0,"En mayo de 1536, justo después de la ejecución..."
241,5.3,Lady Lou es una mujer a finales del siglo XIX ...
242,6.2,John ha llevado una vida solitaria durante tre...


In [114]:
df25 = u.get_properties_film_filmaffinity(oscars.film[244:501], oscars.year_film[244:501])
df25

,Rate,Description
0,6.5,Mickey deberá salvar a su novia Minnie Mouse d...
1,5.3,El conejo Oswald está en el dentista para remo...
2,,
3,,
4,,
...,...,...
252,5.5,Ernie Holley huye en su noche de bodas porque ...
253,7.0,La frontera norte de la India sufre los consta...
254,7.4,Samuel Dodsworth es un magnate de la industria...
255,,Un niño pequeño se convierte en una pieza clav...


In [3]:
df3 = u.get_properties_film_filmaffinity(oscars.film.iloc[501:], oscars.year_film.iloc[501:])
df3

,Rate,Description
0,,
1,5.8,Película sobre la consolidación de la coloniza...
2,,"Nikki Martin, una estrella de la ópera parisin..."
3,5.6,"Tres hermanas (Deanna Durbin, Man Grey y Barba..."
4,,
...,...,...
736,8.2,"En un pueblo minero de Gales viven los Morgan,..."
737,8.1,"A finales del siglo XIX, en una población sure..."
738,8.0,"En el siglo XVI, los Caballeros de la Orden de..."
739,,


In [5]:
list(df2.Rate)

['',
 '7.8',
 '',
 '7.8',
 '6.8',
 '8.4',
 '',
 '7.8',
 '',
 '8.4',
 '8.4',
 '6.0',
 '7.2',
 '7.8',
 '',
 '8.2',
 '',
 '',
 '',
 '6.2',
 '7.8',
 '',
 '8.4',
 '8.2',
 '',
 '6.2',
 '',
 '7.8',
 '7.8',
 '',
 '',
 '',
 '',
 '',
 '5.6',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '6.0',
 '5.2',
 '',
 '',
 '',
 '',
 '7.6',
 '6.5',
 '7.2',
 '5.6',
 '',
 '',
 '',
 '6.0',
 '5.6',
 '',
 '',
 '',
 '',
 '5.6',
 '6.0',
 '5.1',
 '',
 '5.6',
 '',
 '',
 '',
 '6.5',
 '6.9',
 '5.7',
 '',
 '6.5',
 '',
 '',
 '',
 '',
 '',
 '6.6',
 '6.5',
 '',
 '',
 '',
 '6.0',
 '6.6',
 '',
 '',
 '7.8',
 '6.6',
 '',
 '6.6',
 '',
 '6.6',
 '6.5',
 '6.6',
 '7.8',
 '7.0',
 '7.8',
 '6.5',
 '5.7',
 '6.5',
 '6.6',
 '6.5',
 '',
 '6.6',
 '',
 '',
 '7.8',
 '6.5',
 '5.7',
 '6.5',
 '',
 '6.0',
 '',
 '6.5',
 '5.5',
 '6.8',
 '7.0',
 '5.6',
 '6.5',
 '',
 '6.0',
 '6.5',
 '5.4',
 '7.0',
 '6.7',
 '6.2',
 '6.5',
 '7.0',
 '',
 '6.7',
 '7.8',
 '6.5',
 '6.0',
 '6.8',
 '7.0',
 '',
 '6.5',
 '',
 '6.8',
 '',
 '5.4',
 '',
 '',
 '',
 '',
 '6.5',
 '6.8',
 ''

In [8]:
list(df2.Description)

['Cuando su ex mujer se casa con el rico gobernador Bancroft, Buck Gordon ve la oportunidad de un chantaje. Su hijo Nickie, para defender a su madre, planea asesinar a su padre. (FILMAFFINITY)',
 'La Última Orden se inspira en una historia supuestamente real, que se  atribuye a Ernst Lubitsch. Su protagonista es un aristócrata zarista arruinado que, tras la Revolución Soviética, acaba recalando en Hollywood, donde trabaja como extra en una película que narra los convulsos días de la Revolución de 1917, y en la que encarna a un personaje cuya vida es idéntica a la suya. Esta extraña e insólita situación hace que afloren a su memoria los recuerdos del pasado. (FILMAFFINITY)',
 'Inmigrantes húngaros viajan en barco a Estados Unidos en busca de una vida mejor. (FILMAFFINITY)',
 'Chico (Charles Farrell) es un joven que trabaja en la limpieza de las alcantarillas de Paris. Diane (Janet Gaynor) es una chica muy desgraciada que es víctima de los abusos de su atrabiliaria hermana mayor. Un día,

In [4]:
oscars.film.iloc[501:502]

501    San Francisco
Name: film, dtype: object

In [4]:
list(df3.Rate)

['',
 '5.8',
 '',
 '5.6',
 '',
 '6.5',
 '',
 '6.7',
 '5.6',
 '6.7',
 '7.4',
 '7.8',
 '7.7',
 '6.7',
 '',
 '7.0',
 '6.3',
 '7.1',
 '8.1',
 '7.4',
 '6.8',
 '7.1',
 '7.6',
 '5.8',
 '7.4',
 '7.4',
 '7.0',
 '7.3',
 '7.4',
 '6.6',
 '7.6',
 '7.4',
 '',
 '6.3',
 '',
 '6.0',
 '',
 '',
 '7.1',
 '7.6',
 '',
 '7.2',
 '6.5',
 '',
 '5.7',
 '',
 '6.6',
 '7.1',
 '7.6',
 '6.5',
 '7.0',
 '',
 '7.3',
 '',
 '',
 '',
 '7.7',
 '',
 '',
 '6.0',
 '',
 '7.4',
 '7.3',
 '7.1',
 '7.6',
 '7.0',
 '7.4',
 '8.1',
 '7.3',
 '7.6',
 '6.6',
 '6.8',
 '6.6',
 '7.1',
 '7.6',
 '',
 '',
 '6.6',
 '',
 '7.2',
 '6.4',
 '6.8',
 '6.1',
 '6.5',
 '6.7',
 '',
 '',
 '',
 '6.8',
 '',
 '7.4',
 '8.1',
 '',
 '7.3',
 '6.6',
 '7.1',
 '7.6',
 '6.6',
 '7.6',
 '7.0',
 '',
 '5.3',
 '6.9',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '4.1',
 '6.8',
 '6.6',
 '7.1',
 '7.6',
 '',
 '6.6',
 '5.8',
 '5.8',
 '6.4',
 '6.6',
 '7.1',
 '6.6',
 '7.0',
 '7.4',
 '8.1',
 '7.1',
 '7.6',
 '7.0',
 '',
 '6.4',
 '7.5',
 '7.1',
 '7.0',
 '6.9',
 '',
 '6.4',
 '6

In [5]:
list(df3.Description)

['',
 'Película sobre la consolidación de la colonización a través del establecimiento de un cuerpo para mantener la ley, los Rangers. Jim Hawkins y Wahoo Jones son dos asaltadores de diligencias que se dirigen a Texas para buscar a Sam McGee, su compañero. Una vez allí, sin fondos en los bolsillos, se unen a los Rangers de Texas. (FILMAFFINITY)',
 'Nikki Martin, una estrella de la ópera parisina, se marcha en busca de aventuras y amor verdadero, dejando a su esposo dispuesto para estar en el altar. Mientras hace autostop, Nikki conoce al apuesto músico estadounidense, Windy McLean y su banda "McLean Wildcats". Windy la escupe de inmediato, pero Nikki se enamora de él y lo sigue a Nueva York guardando el barco en el que está. El mayordomo la encuentra escondida en Windy y en la habitación de los Wilcats. Ella está encerrada por las autoridades y Windy y la banda son despedidos. Cuando el barco llega a Nueva York, Nikki se escapa del barco y descubre el apartamento de los Wilcats.',
 'T

In [9]:
df4 = u.get_properties_film_filmaffinity(oscars.film.iloc[1241:], oscars.year_film.iloc[1241:])
df4

,Rate,Description
0,6.7,La biografía del bailarín de vodevil George M....
1,7.3,Un oficial (Ronald Colman) que había luchado d...
2,7.0,"Biopic sobre Lou Gehrig, un legendario jugador..."
3,7.2,"Durante la Segunda Guerra Mundial, una familia..."
4,,
...,...,...
669,5.5,"Aladino, un cantante vagabundo, conoce en las ..."
670,6.5,"Edgar, un tímido bibliotecario es persuadido p..."
671,,Susan Darell (Joan Fontaine) a lo largo de su ...
672,6.2,"En 1941, El FBI desenmascaró una red de espion..."


In [10]:
list(df4.Rate)

['6.7',
 '7.3',
 '7.0',
 '7.2',
 '',
 '6.0',
 '7.0',
 '6.7',
 '6.4',
 '7.2',
 '7.6',
 '7.2',
 '7.1',
 '6.3',
 '7.0',
 '7.6',
 '7.7',
 '7.3',
 '7.2',
 '7.2',
 '',
 '7.7',
 '7.0',
 '7.3',
 '6.8',
 '',
 '6.3',
 '6.5',
 '7.2',
 '6.4',
 '5.8',
 '5.7',
 '',
 '5.5',
 '6.6',
 '6.7',
 '7.7',
 '7.2',
 '6.6',
 '',
 '7.0',
 '6.5',
 '7.2',
 '5.5',
 '6.4',
 '5.8',
 '6.8',
 '5.7',
 '',
 '6.6',
 '5.1',
 '6.7',
 '7.2',
 '7.3',
 '6.0',
 '6.7',
 '',
 '5.7',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '6.5',
 '',
 '',
 '',
 '',
 '',
 '5.0',
 '6.4',
 '',
 '',
 '',
 '',
 '',
 '4.6',
 '7.2',
 '7.0',
 '7.2',
 '6.4',
 '6.7',
 '5.8',
 '6.5',
 '6.8',
 '5.9',
 '5.6',
 '',
 '6.6',
 '6.0',
 '',
 '',
 '7.6',
 '7.0',
 '7.3',
 '6.8',
 '',
 '6.5',
 '7.2',
 '8.5',
 '',
 '6.2',
 '6.3',
 '6.6',
 '',
 '5.5',
 '6.7',
 '6.5',
 '',
 '6.5',
 '6.1',
 '',
 '6.3',
 '6.5',
 '',
 '6.8',
 '',
 '6.3',
 '6.7',
 '6.7',
 '7.7',
 '7.2',
 '',
 '7.0',
 '7.3',
 '7.2',
 '6.0',
 '6.7',
 '',
 '6.2',
 '',
 '',
 '6.3',
 '6.0',
 '',
 '',
 '',
 

In [11]:
list(df4.Description)

['La biografía del bailarín de vodevil George M. Cohan, creador del himno norteamericano ”Over There”, sirvió de base para este musical, en el que Cagney demostró sus grandes dotes para el baile. Aunque se trata de una comedia, es una obra de propaganda bélica  realizada en plena Segunda Guerra Mundial (1939-1945). (FILMAFFINITY)',
 'Un oficial (Ronald Colman) que había luchado durante la Primera Guerra Mundial, termina en un hospital psiquiátrico al ser encontrado bajo un estado de amnesia plena. Tras huir de este lugar, es ayudado por la cantante y bailarina Paula (Greer Garson), una mujer bella y generosa que hace cuanto puede para sacarlo adelante. Pero cuando han formalizado sus vidas, Smithy sale con rumbo al periódico El Mercurio, donde espera recibir una empleo fijo... y un nuevo accidente le hará recobrar su vieja memoria, pero olvidará por completo lo que ha vivido en los últimos tres años... incluida Paula. (FILMAFFINITY)',
 'Biopic sobre Lou Gehrig, un legendario jugador pr

In [13]:
df5 = u.get_properties_film_filmaffinity(oscars.film.iloc[1915:], oscars.year_film.iloc[1915:])
df5

,Rate,Description
0,7.2,"Durante la II Guerra Mundial, el Mayor Nelson ..."
1,6.5,Adaptación de la biografía del gran músico pol...
2,6.5,En 1930 John Dillinger era un joven provincian...
3,,
4,4.5,Segunda Guerra Mundial (1939-1945). Historia d...
...,...,...
635,6.7,Un científico amenaza con detonar una bomba at...
636,,
637,7.9,Al estallar la Primera Guerra Mundial (1914-19...
638,7.4,Adaptación de una obra de Broadway que describ...


In [18]:
oscars.film.iloc[2555:2555]

2553    Detective Story
2554           La Ronde
Name: film, dtype: object

3393    The Young Philadelphians
3394     The Diary of Anne Frank
3395                 Pillow Talk
3396              The Nuns Story
3397        Suddenly Last Summer
3398             Room at the Top
3399        Suddenly Last Summer
Name: film, dtype: object

In [14]:
list(df5.Rate)

['7.2',
 '6.5',
 '6.5',
 '',
 '4.5',
 '',
 '',
 '',
 '8.0',
 '7.6',
 '6.3',
 '7.3',
 '8.2',
 '',
 '',
 '6.8',
 '',
 '',
 '',
 '8.1',
 '8.2',
 '',
 '7.4',
 '8.1',
 '7.6',
 '6.6',
 '6.8',
 '7.3',
 '',
 '7.6',
 '5.9',
 '6.4',
 '6.4',
 '',
 '',
 '6.0',
 '',
 '6.8',
 '6.4',
 '',
 '',
 '6.8',
 '8.2',
 '8.1',
 '',
 '7.9',
 '6.8',
 '',
 '',
 '',
 '',
 '',
 '8.2',
 '',
 '',
 '7.9',
 '6.8',
 '6.4',
 '8.2',
 '',
 '7.1',
 '7.9',
 '5.8',
 '5.7',
 '6.0',
 '',
 '6.0',
 '5.7',
 '5.7',
 '6.7',
 '6.0',
 '5.8',
 '8.2',
 '',
 '',
 '',
 '6.8',
 '7.1',
 '',
 '',
 '',
 '6.2',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '8.2',
 '',
 '',
 '6.5',
 '6.9',
 '7.2',
 '7.4',
 '',
 '7.4',
 '',
 '6.5',
 '',
 '8.1',
 '',
 '6.5',
 '6.4',
 '8.2',
 '8.1',
 '7.9',
 '5.6',
 '',
 '6.9',
 '7.7',
 '',
 '6.5',
 '5.8',
 '',
 '6.4',
 '6.9',
 '7.1',
 '7.3',
 '',
 '6.1',
 '',
 '5.8',
 '',
 '6.9',
 '7.1',
 '',
 '6.1',
 '',
 '6.3',
 '7.4',
 '7.4',
 '6.5',
 '7.8',
 '7.4',
 '6.3',
 '7.4',
 '6.5',
 '5.7',
 '',
 '7.1',
 '6.9',
 '',
 '7

In [15]:
list(df5.Description)

['Durante la II Guerra Mundial, el Mayor Nelson y sus hombres son lanzados en paracaídas sobre la jungla birmana, tras las líneas enemigas, para destruir una estación de radar japonesa. La angustiosa y extenuante expedición a través de la jungla y los pantanos repletos de soldados enemigos se convertirá en un juego mortal en el que tan importante será cumplir con éxito la misión como poder ser rescatados... (FILMAFFINITY)',
 'Adaptación de la biografía del gran músico polaco Frederic Chopin. (FILMAFFINITY)',
 'En 1930 John Dillinger era un joven provinciano que iba de pueblo en pueblo en busca de su porvenir en el mundo de los negocios. Cuatro años más tarde termina siendo el criminal más buscado del mundo. (FILMAFFINITY)',
 '',
 'Segunda Guerra Mundial (1939-1945). Historia de dos hermanas. Una de ellas, que está embarazada y forma parte de una orquesta, se preocupa por el destino de su marido en el frente. (FILMAFFINITY)',
 '',
 'Dos soldados que acaban de alistarse son destinados a 

In [19]:
df6 = u.get_properties_film_filmaffinity(oscars.film.iloc[2555:], oscars.year_film.iloc[2555:])
df6

,Rate,Description
0,7.6,"George Eastman (Montgomery Clift), un joven si..."
1,8.0,"Blanche, que pertenece a una rancia pero arrui..."
2,7.3,"Terminada la Segunda Guerra Mundial, Jerry Mul..."
3,8.0,Charles Tatum es un periodista sin escrúpulos ...
4,5.8,Después de matar al gigante Goliath y salir vi...
...,...,...
677,7.6,Un elegante playboy y una bella cantante de un...
678,5.6,"Fedra, una joven pescadora de esponjas del mar..."
679,7.9,"Durante la Segunda Guerra Mundial (1939-1945),..."
680,,A través de la combinación de imágenes reales ...


In [20]:
list(df6.Rate)

['7.6',
 '8.0',
 '7.3',
 '8.0',
 '5.8',
 '5.3',
 '',
 '',
 '7.5',
 '8.1',
 '8.1',
 '7.2',
 '7.3',
 '6.5',
 '7.2',
 '8.0',
 '7.1',
 '7.5',
 '6.9',
 '7.1',
 '',
 '5.8',
 '5.5',
 '8.1',
 '8.1',
 '7.2',
 '6.9',
 '5.5',
 '8.1',
 '',
 '6.5',
 '',
 '7.5',
 '6.0',
 '5.6',
 '7.2',
 '8.0',
 '6.2',
 '8.1',
 '7.2',
 '6.5',
 '5.5',
 '7.1',
 '6.0',
 '7.0',
 '5.4',
 '8.0',
 '6.2',
 '6.2',
 '8.1',
 '',
 '6.5',
 '7.1',
 '6.8',
 '6.0',
 '5.6',
 '7.2',
 '5.5',
 '7.7',
 '6.8',
 '8.1',
 '7.2',
 '8.0',
 '',
 '5.5',
 '',
 '',
 '',
 '',
 '7.0',
 '6.9',
 '',
 '6.8',
 '8.1',
 '7.2',
 '8.1',
 '7.0',
 '5.0',
 '',
 '7.5',
 '6.0',
 '5.9',
 '',
 '8.1',
 '5.5',
 '5.0',
 '',
 '8.1',
 '6.0',
 '',
 '6.8',
 '8.1',
 '7.0',
 '7.2',
 '8.0',
 '6.7',
 '5.7',
 '',
 '5.0',
 '6.0',
 '',
 '',
 '',
 '7.0',
 '',
 '',
 '',
 '',
 '',
 '6.3',
 '6.0',
 '',
 '8.0',
 '5.5',
 '6.1',
 '6.8',
 '6.1',
 '7.1',
 '',
 '6.7',
 '8.1',
 '7.7',
 '8.1',
 '7.1',
 '8.0',
 '',
 '6.3',
 '7.3',
 '6.4',
 '7.5',
 '',
 '7.6',
 '6.3',
 '7.8',
 '7.6',
 '7.8',

In [21]:
list(df6.Description)

['George Eastman (Montgomery Clift), un joven sin recursos, consigue un trabajo gracias a un pariente lejano, el rico industrial Charles Eastman (Herbert Heyes). El empleo es un puesto en su fábrica, pero tan modesto que le impide la entrada en su círculo social. A pesar de ello, el joven conoce a Ángela Vickens (Elizabeth Taylor), una bellísima aristócrata de la que se enamora. Pero George tiene novia, una humilde empleada de la fábrica, razón por la cual tendrá que afrontar un espinoso dilema moral. (FILMAFFINITY)',
 'Blanche, que pertenece a una rancia pero arruinada familia sureña, es una mujer madura y decadente que vive anclada en el pasado. Ciertas circunstancias la obligan a ir a vivir a Nueva Orleáns con su hermana Stella y su cuñado Stanley (Marlon Brando), un hombre rudo y violento. A pesar de su actitud remilgada y arrogante, Blanche oculta un escabroso pasado que la ha conducido al desequilibrio mental. Su inestable conducta provoca conflictos que alteran la vida de la jov

In [24]:
df7 = u.get_properties_film_filmaffinity(oscars.film.iloc[3237:], oscars.year_film.iloc[3237:])
df7

,Rate,Description
0,7.6,Un elegante playboy y una bella cantante de un...
1,5.3,"Joe E. Lewis, un prometedor cantante, pierde l..."
2,,"Nick Conover, un jovencito bastante complicado..."
3,5.4,"Cuando el avión de Pete, un adinerado y joven ..."
4,6.6,"Tras quedarse viudo, un próspero inmigrante qu..."
...,...,...
155,8.0,"Frederick Manion (Ben Gazzara), un teniente de..."
156,6.9,Historia de un joven y ambicioso abogado de Ph...
157,7.0,"Amsterdam, julio de 1942. Con objeto de escapa..."
158,6.8,"Allen, compositor musical, y Jan, decoradora d..."


In [26]:
list(df7.Rate)

['7.6',
 '5.3',
 '',
 '5.4',
 '6.6',
 '7.9',
 '6.8',
 '',
 '8.7',
 '8.6',
 '6.6',
 '6.0',
 '6.0',
 '',
 '5.4',
 '6.2',
 '',
 '',
 '',
 '',
 '7.5',
 '6.8',
 '6.4',
 '',
 '8.6',
 '7.0',
 '6.6',
 '7.9',
 '6.8',
 '6.8',
 '',
 '8.7',
 '7.2',
 '6.9',
 '6.6',
 '7.2',
 '7.7',
 '',
 '7.0',
 '8.0',
 '7.5',
 '7.0',
 '6.7',
 '7.0',
 '6.8',
 '8.0',
 '7.4',
 '',
 '7.3',
 '7.5',
 '7.4',
 '6.9',
 '8.0',
 '6.9',
 '7.5',
 '7.4',
 '6.0',
 '7.0',
 '6.9',
 '6.5',
 '',
 '6.8',
 '8.3',
 '7.0',
 '6.6',
 '7.3',
 '7.5',
 '7.2',
 '6.9',
 '8.0',
 '6.8',
 '6.7',
 '5.9',
 '6.5',
 '5.9',
 '',
 '6.8',
 '7.4',
 '8.0',
 '7.0',
 '6.8',
 '7.3',
 '6.6',
 '',
 '',
 '',
 '4.7',
 '',
 '',
 '',
 '',
 '',
 '6.9',
 '',
 '7.0',
 '6.8',
 '7.3',
 '',
 '',
 '',
 '',
 '',
 '8.0',
 '6.7',
 '7.5',
 '4.7',
 '7.2',
 '',
 '6.1',
 '6.8',
 '',
 '5.9',
 '5.9',
 '',
 '6.8',
 '7.4',
 '6.1',
 '6.9',
 '8.0',
 '7.0',
 '6.8',
 '7.5',
 '6.5',
 '5.6',
 '',
 '',
 '',
 '',
 '',
 '',
 '7.3',
 '5.9',
 '7.6',
 '8.3',
 '7.2',
 '6.2',
 '6.6',
 '8.0',
 '6.

In [27]:
list(df7.Description)

['Un elegante playboy y una bella cantante de un club nocturno se conocen a bordo de un lujoso transatlántico y surge entre ellos un apasionado romance. Aunque ambos están comprometidos (ella es la amante de un magnate y él se va a casar con una rica heredera), establecen un pacto antes de abandonar el barco: encontrarse en el Empire State Building en un plazo de seis meses si siguen sintiendo lo mismo el uno por el otro. (FILMAFFINITY)',
 'Joe E. Lewis, un prometedor cantante, pierde la voz a causa del ataque de unos gángsteres Al verse obligado a cambiar de profesión, decide trabajar como humorista. (FILMAFFINITY)',
 'Nick Conover, un jovencito bastante complicado, debe de salir de Chicago tras ser liberado de la cárcel por robar un coche. Así que sus padres deciden enviarlo a la granja de Kentucky de su tío Jed Bruce y su tía Enriqueta. Allí en el fin del mundo iniciará una vida feliz, llena de amigos y hasta un posible amor. (FILMAFFINITY)',
 'Cuando el avión de Pete, un adinerado 

In [30]:
df7 = u.get_properties_film_filmaffinity(oscars.film.iloc[3397:], oscars.year_film.iloc[3397:])
df7

,Rate,Description
0,7.6,"En la Nueva Orleans de 1937, una rica viuda, l..."
1,7.5,Joe Lampton (Laurence Harvey) acepta un trabaj...
2,7.6,"En la Nueva Orleans de 1937, una rica viuda, l..."
3,7.5,Joe Lampton (Laurence Harvey) acepta un trabaj...
4,7.6,"Lora Meredith (Lana Turner), una actriz viuda ..."
...,...,...
720,,
721,7.3,Una joven ciega dominada por su madre conoce u...
722,6.2,Antes de la Segunda Guerra Mundial (1939-1945)...
723,6.0,Narra las tribulaciones de una joven estrella ...


In [32]:
import json

df7_dict = [{"Rate7": list(df7.Rate)}, {"Description7":list(df7.Description)}]
with open('prueba_json.json', 'w') as json_file:
    json.dump(df7_dict, json_file)

In [33]:
df65 = u.get_properties_film_filmaffinity(oscars.film.iloc[3237:3398], oscars.year_film.iloc[3237:3398])
df65

,Rate,Description
0,7.6,Un elegante playboy y una bella cantante de un...
1,5.3,"Joe E. Lewis, un prometedor cantante, pierde l..."
2,,"Nick Conover, un jovencito bastante complicado..."
3,5.4,"Cuando el avión de Pete, un adinerado y joven ..."
4,6.6,"Tras quedarse viudo, un próspero inmigrante qu..."
...,...,...
156,6.9,Historia de un joven y ambicioso abogado de Ph...
157,7.0,"Amsterdam, julio de 1942. Con objeto de escapa..."
158,6.8,"Allen, compositor musical, y Jan, decoradora d..."
159,,


In [38]:
total_rate = list(df3.Rate) + list(df4.Rate) + list(df5.Rate) + list(df6.Rate) + list(df65.Rate) + list(df7.Rate)
total_description = list(df3.Description) + list(df4.Description) + list(df5.Description) + list(df6.Description) + list(df65.Description) + list(df7.Description)
len(total_rate)
len(total_description)

3623

In [41]:
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [45]:
df8 = u.get_properties_film_filmaffinity(oscars.film.iloc[4122:], oscars.year_film.iloc[4122:])
df8

,Rate,Description
0,6.9,"Adaptación del drama homónimo de Shakespeare, ..."
1,7.3,Una joven ciega dominada por su madre conoce u...
2,6.9,"Austria, 1938. María es una alegre novicia que..."
3,6.9,"Durante la Segunda Guerra Mundial (1939-1945),..."
4,7.3,Una joven ciega dominada por su madre conoce u...
...,...,...
117,,
118,7.8,"Durante la II Guerra Mundial (1939-1945), en u..."
119,6.1,"Georgy es una mujer soltera, poco atractiva, p..."
120,,


In [46]:
total_rate += list(df8.Rate)
total_description += list(df8.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [52]:
df9 = u.get_properties_film_filmaffinity(oscars.film.iloc[4244:], oscars.year_film.iloc[4244:])
df9

,Rate,Description
0,,
1,7.5,Para divorciarse de su esposa Catalina de Arag...
2,,
3,6.4,"Alfie Elkins (Michael Caine), un 'cockney' lon..."
4,,
...,...,...
494,6.9,"Adaptación musical del popular ""Cuento de Navi..."
495,5.3,Mike Vecchio y Susan Henderson están ultimando...
496,,El Padre Gregory Lind (Robert Forster) es un s...
497,6.9,"Adaptación musical del popular ""Cuento de Navi..."


In [53]:
total_rate += list(df9.Rate)
total_description += list(df9.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [55]:
df9 = u.get_properties_film_filmaffinity(oscars.film.iloc[4743:], oscars.year_film.iloc[4743:])
df9

,Rate,Description
0,5.9,"Durante la I Guerra Mundial (1914-1918), una p..."
1,5.8,"Un temporal de nieve azota Chicago, dificultan..."
2,6.8,"Robert Dupea, que fue un niño prodigio como pi..."
3,6.4,Historia de amor de dos universitarios de Harv...
4,,
...,...,...
709,5.3,Narra la historia real del secuestro de un avi...
710,,
711,,"Jalal Toufic, tras haber tratado de colaborar ..."
712,7.3,"Una noche, cerca de su casa, en Indiana, Roy N..."


In [121]:
len(list(df9.Rate))

714

In [56]:
total_rate += list(df9.Rate)
total_description += list(df9.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [62]:
df11 = u.get_properties_film_filmaffinity(oscars.film.iloc[5457:], oscars.year_film.iloc[5457:])
df11

,Rate,Description
0,,"Documental sobre la realización de 'Mahoma, el..."
1,6.1,La misteriosa desaparición de dos submarinos n...
2,7.9,"La princesa Leia, líder del movimiento rebelde..."
3,5.2,Musical inspirado en la película de Ingmar Ber...
4,,
...,...,...
343,,
344,8.1,"A finales del siglo XIX, el doctor Frederick T..."
345,7.2,Conrad acaba de salir del hospital después de ...
346,5.8,"Cameron (Steve Railsback), un convicto persegu..."


In [63]:
total_rate += list(df11.Rate)
total_description += list(df11.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [66]:
df12 = u.get_properties_film_filmaffinity(oscars.film.iloc[5805:], oscars.year_film.iloc[5805:])
df12

,Rate,Description
0,6.3,Un grupo de jóvenes músicos y bailarines se pr...
1,6.0,"Melvin recoge en el desierto de Tonopah, en un..."
2,,
3,4.6,"Tras morir su marido en la noche de bodas, Jud..."
4,8.1,Tras un ataque sorpresa de las tropas imperial...
...,...,...
727,5.7,En la pequeña y tranquila población de Eastwic...
728,6.3,El temperamental piloto Tuck Pendelton (Dennis...
729,6.8,Un grupo de mercenarios es contratado por la C...
730,,


In [67]:
total_rate += list(df12.Rate)
total_description += list(df12.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [70]:
df13 = u.get_properties_film_filmaffinity(oscars.film.iloc[6537:], oscars.year_film.iloc[6537:])
df13

,Rate,Description
0,8.2,Un grupo de reclutas se prepara en Parris Isla...
1,7.2,"Pekín, 1908. China se encuentra en plena decad..."
2,7.2,Narra la odisea de un niño de doce años a quie...
3,,
4,6.3,En la redacción de informativos de una cadena ...
...,...,...
222,6.6,"A principios de los años 80, un pequeño círcul..."
223,7.8,"Michael Corleone, heredero del imperio de don ..."
224,7.4,Tras la Guerra de Secesión (1861-1865) y en pl...
225,5.4,Dick Tracy (Warren Beatty) es un policía valie...


In [71]:
total_rate += list(df13.Rate)
total_description += list(df13.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [74]:
df14 = u.get_properties_film_filmaffinity(oscars.film.iloc[6764:], oscars.year_film.iloc[6764:])
df14

,Rate,Description
0,7.5,Un escritor llamado Paul Sheldon (James Caan) ...
1,6.6,"Roy Dillon (John Cusack), hijo de una prostitu..."
2,6.3,"Edward Lewis (Richard Gere), un apuesto y rico..."
3,5.7,Hollywood. Tras salir de una clínica de desint...
4,6.0,"En los años cuarenta, los Bridge, un rico matr..."
...,...,...
319,,
320,7.0,"La vida del doctor Richard Kimble, un reputado..."
321,6.7,Frank Horrigan (Clint Eastwood) es un veterano...
322,8.2,"Belfast, años 70. Gerry (Day-Lewis) es un gamb..."


In [75]:
total_rate += list(df14.Rate)
total_description += list(df14.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [77]:
df15 = u.get_properties_film_filmaffinity(oscars.film.iloc[7088:], oscars.year_film.iloc[7088:])
df15

,Rate,Description
0,6.3,Paul consigue introducirse hábilmente en el mu...
1,7.4,"Año 1851. Ada, que es muda desde niña, acaba d..."
2,7.6,"En 1958, Stevens (Anthony Hopkins), un perfect..."
3,7.4,"C.S. Lewis (Anthony Hopkins), profesor de lite..."
4,6.6,"Mitch McDeere (Tom Cruise), un joven y brillan..."
...,...,...
688,6.8,Sarah Miles es una mujer apasionada y atrapada...
689,7.2,"Jeffrey Wigand, científico y directivo de la f..."
690,7.0,"Norteamérica, finales del siglo XVIII. El cond..."
691,6.3,Durante el juicio en el que un japonés es acus...


In [78]:
total_rate += list(df15.Rate)
total_description += list(df15.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [90]:
len(total_rate)

7281

In [84]:
list(df15.Rate)

['6.3',
 '7.4',
 '7.6',
 '7.4',
 '6.6',
 '7.4',
 '',
 '7.0',
 '8.2',
 '5.4',
 '7.0',
 '6.4',
 '7.6',
 '',
 '7.3',
 '7.0',
 '7.4',
 '',
 '6.8',
 '7.0',
 '6.4',
 '7.4',
 '7.6',
 '',
 '8.2',
 '7.4',
 '7.6',
 '',
 '7.5',
 '',
 '',
 '',
 '',
 '7.0',
 '',
 '',
 '',
 '7.0',
 '6.7',
 '8.2',
 '7.4',
 '',
 '6.7',
 '7.3',
 '',
 '',
 '7.1',
 '5.6',
 '7.4',
 '',
 '7.0',
 '6.6',
 '7.0',
 '7.6',
 '',
 '4.9',
 '',
 '7.4',
 '7.4',
 '6.0',
 '7.0',
 '8.2',
 '7.4',
 '7.6',
 '',
 '',
 '6.6',
 '',
 '',
 '7.4',
 '6.8',
 '',
 '',
 '6.0',
 '',
 '5.4',
 '7.0',
 '5.5',
 '7.0',
 '',
 '5.4',
 '7.0',
 '7.0',
 '5.4',
 '7.0',
 '7.7',
 '7.0',
 '8.2',
 '7.6',
 '',
 '7.4',
 '',
 '6.7',
 '7.4',
 '7.4',
 '6.0',
 '',
 '8.6',
 '8.2',
 '6.4',
 '',
 '8.6',
 '8.6',
 '7.5',
 '7.5',
 '6.9',
 '8.2',
 '5.8',
 '',
 '5.7',
 '6.2',
 '6.5',
 '5.7',
 '6.4',
 '8.6',
 '7.5',
 '7.5',
 '7.5',
 '8.2',
 '7.2',
 '6.8',
 '6.4',
 '8.2',
 '6.8',
 '',
 '8.6',
 '6.3',
 '6.7',
 '7.5',
 '6.2',
 '6.1',
 '',
 '7.5',
 '8.2',
 '8.6',
 '6.9',
 '',
 '',
 

In [91]:
df16 = u.get_properties_film_filmaffinity(oscars.film.iloc[7780:], oscars.year_film.iloc[7780:])
df16

,Rate,Description
0,5.9,"Tailandia, finales del siglo XIX. La vida de l..."
1,7.0,"Norteamérica, finales del siglo XVIII. El cond..."
2,6.8,Adaptación de la novela homónima de Patricia H...
3,5.6,El gran general romano Titus Andrónicus regres...
4,6.5,Drama musical ambientado en el siglo XIX. Narr...
...,...,...
595,7.2,"Historia basada en la vida de James Barrie, el..."
596,8.0,Después de haber entrenado y representado a lo...
597,7.1,La vida de Ray Charles es una alternancia de t...
598,6.7,"Miles, un divorciado deprimido, aspirante a es..."


In [92]:
total_rate += list(df16.Rate)
total_description += list(df16.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [94]:
df17 = u.get_properties_film_filmaffinity(oscars.film.iloc[8380:], oscars.year_film.iloc[8380:])
df17

,Rate,Description
0,5.6,La historia nos muestra a una ardilla de lo má...
1,6.6,"En ""Guard Dog"", Bil Plympton nos introduce en ..."
2,,
3,,
4,,
...,...,...
579,7.8,Segunda Guerra Mundial (1939-1945). En la Fran...
580,6.8,"Remade de la mítica saga de Star Trek, dirigid..."
581,7.9,Carl Fredricksen es un viudo vendedor de globo...
582,7.2,"Año 2154. Jake Sully (Sam Worthington), un ex-..."


In [95]:
total_rate += list(df17.Rate)
total_description += list(df17.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [98]:
df18 = u.get_properties_film_filmaffinity(oscars.film.iloc[8964:], oscars.year_film.iloc[8964:])
df18

,Rate,Description
0,7.8,Segunda Guerra Mundial (1939-1945). En la Fran...
1,6.8,"Remade de la mítica saga de Star Trek, dirigid..."
2,5.2,Han pasado dos años desde que el joven Sam Wit...
3,7.2,"Año 2154. Jake Sully (Sam Worthington), un ex-..."
4,6.8,Tras la llegada de una enorme nave espacial a ...
...,...,...
546,6.6,Durante las últimas semanas caóticas de la Gue...
547,8.1,"Desde hace cuarenta años, el fotógrafo Sebasti..."
548,7.8,Un grupo de personas valientes arriesgan sus v...
549,,Documental sobre los voluntarios de atención t...


In [99]:
total_rate += list(df18.Rate)
total_description += list(df18.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [101]:
df19 = u.get_properties_film_filmaffinity(oscars.film.iloc[9515:], oscars.year_film.iloc[9515:])
df19

,Rate,Description
0,,
1,6.6,
2,,Miles de desesperadas almas se dirigen hacia l...
3,6.4,"Autobiografía del marine SEAL Chris Kyle, un t..."
4,7.3,Historia dramática que recorre 12 años (2002-2...
...,...,...
174,6.8,En la zona fronteriza que se extiende entre Es...
175,6.8,Treinta años después de la victoria de la Alia...
176,6.8,"James Donovan (Tom Hanks), un abogado de Brook..."
177,7.1,"Perseguido por su turbulento pasado, Mad Max c..."


In [102]:
total_rate += list(df19.Rate)
total_description += list(df19.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [106]:
total_rate = total_rate[:-1]
total_description = total_description[:-1]

In [108]:
df20 = u.get_properties_film_filmaffinity(oscars.film.iloc[9693:], oscars.year_film.iloc[9693:])
df20

,Rate,Description
0,7.0,"Durante una misión tripulada a Marte, el astro..."
1,7.2,Año 1823. En las profundidades de la América s...
2,6.8,Treinta años después de la victoria de la Alia...
3,7.1,"Nathan, un programador multimillonario con fam..."
4,7.1,"Perseguido por su turbulento pasado, Mad Max c..."
...,...,...
380,6.3,El encuentro con un activista medioambiental y...
381,7.6,Años 60. Tony Lip (Viggo Mortensen) es un rudo...
382,7.1,Cleo (Yalitza Aparicio) es la joven sirvienta ...
383,6.9,Explora la historia real sobre cómo Dick Chene...


In [109]:
total_rate += list(df20.Rate)
total_description += list(df20.Description)
len(total_rate)
len(total_description)
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [ ]:
df87 = u.get_properties_film_filmaffinity(oscars.film.iloc[9693:], oscars.year_film.iloc[9693:])
df87

In [112]:
oscars.film.iloc[10076:11000]

10076    Vice
10077        
Name: film, dtype: object

In [115]:
a = [1, 2]
b = [3, 4]
c = a + b
d = b + a
print(c)
print(d)

[1, 2, 3, 4]
[3, 4, 1, 2]


In [116]:
total_rate3 = list(df2.Rate) + list(df25.Rate) + total_rate
total_description3 =list(df2.Description) + list(df25.Description) + total_description

In [117]:
len(total_rate3)

10080

In [118]:
len(total_description3)

10080

In [119]:
len(list(oscars.film))

10078

In [175]:
df85 = u.get_properties_film_filmaffinity(oscars.film.iloc[4652:4744], oscars.year_film.iloc[4652:4744])
df85

,Rate,Description
0,7.9,Un grupo de jóvenes pistoleros se dedica a asa...
1,5.4,Ben Harvey (Beau Bridges) es un campesino que ...
2,6.8,"Cornelius Hackl, dependiente de una tienda de ..."
3,5.6,"Debido a un fallo mecánico, durante una misión..."
4,5.3,"A finales del siglo XIX, el barco ""Batavia Que..."
...,...,...
87,5.3,Mike Vecchio y Susan Henderson están ultimando...
88,,El Padre Gregory Lind (Robert Forster) es un s...
89,6.9,"Adaptación musical del popular ""Cuento de Navi..."
90,4.3,Una monja es la única superviviente de una mat...


In [ ]:
4244-4651

In [178]:
df84 = u.get_properties_film_filmaffinity(oscars.film.iloc[4244:4652], oscars.year_film.iloc[4244:4652])
df84

,Rate,Description
0,,
1,7.5,Para divorciarse de su esposa Catalina de Arag...
2,,
3,6.4,"Alfie Elkins (Michael Caine), un 'cockney' lon..."
4,,
...,...,...
403,,
404,,
405,,
406,,


In [245]:
oscars.film.iloc[4743]

'Darling Lili'

In [246]:
df9

,Rate,Description
0,5.9,"Durante la I Guerra Mundial (1914-1918), una p..."
1,5.8,"Un temporal de nieve azota Chicago, dificultan..."
2,6.8,"Robert Dupea, que fue un niño prodigio como pi..."
3,6.4,Historia de amor de dos universitarios de Harv...
4,,
...,...,...
709,5.3,Narra la historia real del secuestro de un avi...
710,,
711,,"Jalal Toufic, tras haber tratado de colaborar ..."
712,7.3,"Una noche, cerca de su casa, en Indiana, Roy N..."


In [187]:
#2==>0-243(244)
len(list(df2.Rate))

244

In [188]:
#25==>244-500(257)
len(list(df25.Rate))

257

In [189]:
#3==>501-1241(741)
len(list(df3.Rate))

741

In [190]:
#4==>1242-1914(673)
#4==>1241-1914(674)
len(list(df4.Rate))

674

In [191]:
#5==>1915-2554(640)
len(list(df5.Rate))

640

In [192]:
#6==>2555-3236(682)
len(list(df6.Rate))

682

In [193]:
#65==>3237-3397(161)
len(list(df65.Rate))

161

In [194]:
#7==>3398-4121(724)
#7==>3397-4121(725)
len(list(df7.Rate))

725

In [195]:
#8==>4122-4243(122)
len(list(df8.Rate))

122

In [198]:
#84==>4244-4651(408)
len(list(df84.Rate))

408

In [199]:
#85==>4652-4743(92)
len(list(df85.Rate))

92

In [200]:
#9==>4743-5456(714)
len(list(df9.Rate))

714

In [201]:
#11==>5457-5804(348)
len(list(df11.Rate))

348

In [202]:
#12==>5805-6536(732)
len(list(df12.Rate))

732

In [203]:
#13==>6537-6763(227)
len(list(df13.Rate))

227

In [204]:
#14==>6764-7087(324)
len(list(df14.Rate))

324

In [205]:
#15==>7088-7779(692)
#15==>7088-7780(693)
len(list(df15.Rate))

693

In [206]:
#16==>7780-8379(600)
len(list(df16.Rate))

600

In [207]:
#17==>8380-8963(584)
len(list(df17.Rate))

584

In [208]:
#18==>8964-9514(551)
len(list(df18.Rate))

551

In [209]:
#19==>9515-9692(178)
#19==>9515-9693(179)
len(list(df19.Rate))

179

In [210]:
#20==>9693-10077(385)
len(list(df20.Rate))

385

In [169]:
a = len(list(df2.Rate))+len(list(df25.Rate))

In [163]:
b = len(list(df3.Rate))+len(list(df4.Rate))+len(list(df5.Rate))+len(list(df6.Rate))+len(list(df65.Rate))
b

2898

In [164]:
c = len(list(df7.Rate))+len(list(df8.Rate))+len(list(df9.Rate))+len(list(df11.Rate))+len(list(df12.Rate))
c

2641

In [165]:
d = len(list(df13.Rate))+len(list(df14.Rate))+len(list(df15.Rate))+len(list(df16.Rate))+len(list(df17.Rate))
d

2428

In [166]:
e = len(list(df18.Rate))+len(list(df19.Rate))+len(list(df20.Rate))
e

1115

In [172]:
a+b+c+d+e+len(list(df85.Rate))

9991

In [256]:
total_rate = list(df2.Rate) + list(df25.Rate)+list(df3.Rate[:-1])+list(df4.Rate)
total_description = list(df2.Description) + list(df25.Description)+list(df3.Description[:-1])+list(df4.Description)

In [257]:
total_rate += list(df5.Rate) + list(df6.Rate) + list(df65.Rate) + list(df7.Rate[1:])
total_description += list(df5.Description) + list(df6.Description) + list(df65.Description) + list(df7.Description[1:])

In [258]:
total_rate += list(df8.Rate) + list(df84.Rate) + list(df85.Rate) + list(df9.Rate[1:])
total_description += list(df8.Description) + list(df84.Description) + list(df85.Description) + list(df9.Description[1:])

In [259]:
total_rate += list(df11.Rate) + list(df12.Rate) + list(df13.Rate) + list(df14.Rate) + list(df15.Rate[:-1])
total_description += list(df11.Description) + list(df12.Description) + list(df13.Description) + list(df14.Description) + list(df15.Description[:-1])

In [260]:
total_rate += list(df16.Rate) + list(df17.Rate) + list(df18.Rate) + list(df19.Rate[:-1]) + list(df20.Rate)
total_description += list(df16.Description) + list(df17.Description) + list(df18.Description) + list(df19.Description[:-1]) + list(df20.Description)

In [261]:
print(len(total_rate))
len(total_description)

10078


10078

In [255]:
len(oscars.film)

10078

In [262]:
filmaffinity_dict = {"Rate": total_rate}, {"Description":total_description}
with open('prueba_json.json', 'w') as json_file:
    json.dump(filmaffinity_dict, json_file)

In [271]:
with open('prueba_json.json') as json_file:
    film_json = json.load(json_file)


In [275]:
data = {'Rate': film_json[0]["Rate"], 'Description': film_json[1]["Description"]}
filmdf = pd.DataFrame(data = data)
filmdf

,Rate,Description
0,,Cuando su ex mujer se casa con el rico goberna...
1,7.8,La Última Orden se inspira en una historia sup...
2,,Inmigrantes húngaros viajan en barco a Estados...
3,7.8,Chico (Charles Farrell) es un joven que trabaj...
4,6.8,"A la isla Pago-Pago en el Pacífico, llegan sim..."
...,...,...
10073,6.3,El encuentro con un activista medioambiental y...
10074,7.6,Años 60. Tony Lip (Viggo Mortensen) es un rudo...
10075,7.1,Cleo (Yalitza Aparicio) es la joven sirvienta ...
10076,6.9,Explora la historia real sobre cómo Dick Chene...


In [277]:
df = u.get_properties_film(oscars.film, oscars.year_film, oscars, properties=["original_title", "release_date", "genre_ids", "poster_path"])

In [278]:
df

,original_title,release_date,genre_ids,poster_path
0,The Noose,1928-01-29,[18],/hheIsv2RAp8htWZspOPPEcEdlTL.jpg
1,The Last Command,1928-01-21,"[18, 36, 10749, 10752]",/kChMDyZ4EL6cERFSnCeIzG1fSGj.jpg
2,A Ship Comes In,1928-01-04,[18],/tvAxJPepmiN8HUxAQiQamHKeeRD.jpg
3,7th Heaven,1927-01-01,"[18, 10749]",/meu6Y6yL506kJ8ke1ZVItYK4ezR.jpg
4,Sadie Thompson,1928-01-07,"[18, 10749]",/aqNKiFC7cqq6Z2XlFWuLwyjvooK.jpg
...,...,...,...,...
10073,First Reformed,2018-05-18,[18],/p2UylV0hPefEuWPFMo1r56vu2nb.jpg
10074,Green Book,2018-11-16,"[35, 18]",/7BsvSuDQuoqhWmU2fL7W2GOcZHU.jpg
10075,Roma,2018-08-25,[18],/dtIIyQyALk57ko5bjac7hi01YQ.jpg
10076,Vice,2018-12-25,"[35, 18, 36]",/seqLrPNbWJdcX6PeEhBNkmxjz6h.jpg


In [279]:
dict = {"original_title": list(df.original_title)}, {"release_date":list(df.release_date)}, {"genre_ids":list(df.genre_ids) }, {"poster_path": list(df.poster_path)}
with open('TheMovieDB_json.json', 'w') as json_file:
    json.dump(dict, json_file)

In [280]:
oscars = pd.read_csv("./INPUT/the_oscar_award.csv")

In [288]:
with open('src/TheMovieDB_json.json') as json_file:
    TheMovieDB_json = json.load(json_file)
TheMovieDB_json[0]

{'original_title': ['The Noose',
  'The Last Command',
  'A Ship Comes In',
  '7th Heaven',
  'Sadie Thompson',
  'Sunrise: A Song of Two Humans',
  'The Dove',
  '7th Heaven',
  'The Devil Dancer',
  'Sunrise: A Song of Two Humans',
  'Sunrise: A Song of Two Humans',
  'Two Arabian Knights',
  'Speedy',
  '7th Heaven',
  'Sorrell and Son',
  'The Crowd',
  '',
  'Wings',
  '',
  'The Racket',
  '7th Heaven',
  'Wings',
  'Sunrise: A Song of Two Humans',
  'The Crowd',
  'Chang: A Drama of the Wilderness',
  'The Jazz Singer',
  'Glorious Betsy',
  '7th Heaven',
  'The Last Command',
  'Underworld',
  'The Private Life of Helen of Troy',
  '',
  '',
  'Thunderbolt',
  'In Old Arizona',
  'Alibi',
  'The Valiant',
  'The Patriot',
  'Madame X',
  'The Barker',
  'The Letter',
  'The Divine Lady',
  'The Broadway Melody',
  'Coquette',
  'The Patriot',
  'The Bridge of San Luis Rey',
  'Dynamite',
  'Alibi',
  'Street Angel',
  'Our Dancing Daughters',
  'White Shadows in the South Seas'

In [290]:
data = {'Original_Title': TheMovieDB_json[0]["original_title"], 'Release_Date': TheMovieDB_json[1]["release_date"],'Genre_Ids': TheMovieDB_json[2]["genre_ids"], 'Poster_Path': TheMovieDB_json[3]["poster_path"] }


In [291]:
TheMovieDB_df = pd.DataFrame(data = data)

In [295]:
TheMovieDB_df["Genres"] = TheMovieDB_df.Genre_Ids.apply(u.get_genre)

In [296]:
TheMovieDB_df["Genres"]

0                               [Drama]
1        [Drama, History, Romance, War]
2                               [Drama]
3                      [Drama, Romance]
4                      [Drama, Romance]
                      ...              
10073                           [Drama]
10074                   [Comedy, Drama]
10075                           [Drama]
10076          [Comedy, Drama, History]
10077                                []
Name: Genres, Length: 10078, dtype: object

In [297]:
with open('src/filmaff_json.json') as json_file:
    film_json = json.load(json_file)


In [298]:
data = {'Rate': film_json[0]["Rate"], 'Description': film_json[1]["Description"]}
film_df = pd.DataFrame(data = data)

In [306]:
oscars["Genres"] = TheMovieDB_df["Genres"]
oscars["Poster"] = TheMovieDB_df["Poster_Path"]
oscars["Description"] = film_df["Description"]
oscars["Rate"] = film_df["Rate"]
#El df final con el que trabajaremos será:
oscars_final = oscars[["film", "year_film", "category", "name", "win", "Genres", "Rate","Description", "Poster"]]


In [308]:
type(oscars_final.columns)

pandas.core.indexes.base.Index

In [310]:
u.get_genresId_dict()

{28: 'Action',
 12: 'Adventure',
 16: 'Animation',
 35: 'Comedy',
 80: 'Crime',
 99: 'Documentary',
 18: 'Drama',
 10751: 'Family',
 14: 'Fantasy',
 36: 'History',
 27: 'Horror',
 10402: 'Music',
 9648: 'Mystery',
 10749: 'Romance',
 878: 'Science Fiction',
 10770: 'TV Movie',
 53: 'Thriller',
 10752: 'War',
 37: 'Western'}

In [313]:
type(oscars_final.win[4])

numpy.bool_

In [314]:
oscars_final

,film,year_film,category,name,win,Genres,Rate,Description,Poster
0,The Noose,1927,ACTOR,Richard Barthelmess,False,[Drama],,Cuando su ex mujer se casa con el rico goberna...,/hheIsv2RAp8htWZspOPPEcEdlTL.jpg
1,The Last Command,1927,ACTOR,Emil Jannings,True,"[Drama, History, Romance, War]",7.8,La Última Orden se inspira en una historia sup...,/kChMDyZ4EL6cERFSnCeIzG1fSGj.jpg
2,A Ship Comes In,1927,ACTRESS,Louise Dresser,False,[Drama],,Inmigrantes húngaros viajan en barco a Estados...,/tvAxJPepmiN8HUxAQiQamHKeeRD.jpg
3,7th Heaven,1927,ACTRESS,Janet Gaynor,True,"[Drama, Romance]",7.8,Chico (Charles Farrell) es un joven que trabaj...,/meu6Y6yL506kJ8ke1ZVItYK4ezR.jpg
4,Sadie Thompson,1927,ACTRESS,Gloria Swanson,False,"[Drama, Romance]",6.8,"A la isla Pago-Pago en el Pacífico, llegan sim...",/aqNKiFC7cqq6Z2XlFWuLwyjvooK.jpg
...,...,...,...,...,...,...,...,...,...
10073,First Reformed,2018,WRITING (Original Screenplay),Written by Paul Schrader,False,[Drama],6.3,El encuentro con un activista medioambiental y...,/p2UylV0hPefEuWPFMo1r56vu2nb.jpg
10074,Green Book,2018,WRITING (Original Screenplay),Written by Nick Vallelonga Brian Currie Peter ...,True,"[Comedy, Drama]",7.6,Años 60. Tony Lip (Viggo Mortensen) es un rudo...,/7BsvSuDQuoqhWmU2fL7W2GOcZHU.jpg
10075,Roma,2018,WRITING (Original Screenplay),Written by Alfonso Cuarón,False,[Drama],7.1,Cleo (Yalitza Aparicio) es la joven sirvienta ...,/dtIIyQyALk57ko5bjac7hi01YQ.jpg
10076,Vice,2018,WRITING (Original Screenplay),Written by Adam McKay,False,"[Comedy, Drama, History]",6.9,Explora la historia real sobre cómo Dick Chene...,/seqLrPNbWJdcX6PeEhBNkmxjz6h.jpg


In [338]:
result = oscars_final

In [339]:
result.Genres = result.Genres.apply(lambda x: ",".join(x))

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [340]:
result

,film,year_film,category,name,win,Genres,Rate,Description,Poster
0,The Noose,1927,ACTOR,Richard Barthelmess,False,Drama,,Cuando su ex mujer se casa con el rico goberna...,/hheIsv2RAp8htWZspOPPEcEdlTL.jpg
1,The Last Command,1927,ACTOR,Emil Jannings,True,"Drama,History,Romance,War",7.8,La Última Orden se inspira en una historia sup...,/kChMDyZ4EL6cERFSnCeIzG1fSGj.jpg
2,A Ship Comes In,1927,ACTRESS,Louise Dresser,False,Drama,,Inmigrantes húngaros viajan en barco a Estados...,/tvAxJPepmiN8HUxAQiQamHKeeRD.jpg
3,7th Heaven,1927,ACTRESS,Janet Gaynor,True,"Drama,Romance",7.8,Chico (Charles Farrell) es un joven que trabaj...,/meu6Y6yL506kJ8ke1ZVItYK4ezR.jpg
4,Sadie Thompson,1927,ACTRESS,Gloria Swanson,False,"Drama,Romance",6.8,"A la isla Pago-Pago en el Pacífico, llegan sim...",/aqNKiFC7cqq6Z2XlFWuLwyjvooK.jpg
...,...,...,...,...,...,...,...,...,...
10073,First Reformed,2018,WRITING (Original Screenplay),Written by Paul Schrader,False,Drama,6.3,El encuentro con un activista medioambiental y...,/p2UylV0hPefEuWPFMo1r56vu2nb.jpg
10074,Green Book,2018,WRITING (Original Screenplay),Written by Nick Vallelonga Brian Currie Peter ...,True,"Comedy,Drama",7.6,Años 60. Tony Lip (Viggo Mortensen) es un rudo...,/7BsvSuDQuoqhWmU2fL7W2GOcZHU.jpg
10075,Roma,2018,WRITING (Original Screenplay),Written by Alfonso Cuarón,False,Drama,7.1,Cleo (Yalitza Aparicio) es la joven sirvienta ...,/dtIIyQyALk57ko5bjac7hi01YQ.jpg
10076,Vice,2018,WRITING (Original Screenplay),Written by Adam McKay,False,"Comedy,Drama,History",6.9,Explora la historia real sobre cómo Dick Chene...,/seqLrPNbWJdcX6PeEhBNkmxjz6h.jpg


In [367]:
import sys
t = ["Drama" in e for e in result.Genres]
d = np.array([t,]*9).transpose()
np.set_printoptions(threshold=sys.maxsize)
display(d)

result np.array([t,]*9).transpose()

SyntaxError: invalid syntax (<ipython-input-367-1fa19d48f870>, line 7)

In [365]:
t

[True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 T

In [356]:
t

[True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 T

In [333]:
result

,film,year_film,category,name,win,Genres,Rate,Description,Poster


In [327]:

result

,film,year_film,category,name,win,Genres,Rate,Description,Poster
9956,Vice,2018,ACTOR IN A LEADING ROLE,Christian Bale,False,"[Comedy, Drama, History]",6.9,Explora la historia real sobre cómo Dick Chene...,/seqLrPNbWJdcX6PeEhBNkmxjz6h.jpg
9957,A Star Is Born,2018,ACTOR IN A LEADING ROLE,Bradley Cooper,False,"[Drama, Music, Romance]",6.6,Jackson Maine (Bradley Cooper) es una estrella...,/wrFpXMNBRj2PBiN4Z5kix51XaIZ.jpg
9958,At Eternitys Gate,2018,ACTOR IN A LEADING ROLE,Willem Dafoe,False,[Drama],,,/7rUnZrcSyfwfloeI5aoccztSLSg.jpg
9959,Bohemian Rhapsody,2018,ACTOR IN A LEADING ROLE,Rami Malek,True,"[Drama, Music]",7.3,'Bohemian Rhapsody' es una celebración del gru...,/lHu1wtNaczFPGFDTrjCSzeLPTKN.jpg
9960,Green Book,2018,ACTOR IN A LEADING ROLE,Viggo Mortensen,False,"[Comedy, Drama]",7.6,Años 60. Tony Lip (Viggo Mortensen) es un rudo...,/7BsvSuDQuoqhWmU2fL7W2GOcZHU.jpg
...,...,...,...,...,...,...,...,...,...
10073,First Reformed,2018,WRITING (Original Screenplay),Written by Paul Schrader,False,[Drama],6.3,El encuentro con un activista medioambiental y...,/p2UylV0hPefEuWPFMo1r56vu2nb.jpg
10074,Green Book,2018,WRITING (Original Screenplay),Written by Nick Vallelonga Brian Currie Peter ...,True,"[Comedy, Drama]",7.6,Años 60. Tony Lip (Viggo Mortensen) es un rudo...,/7BsvSuDQuoqhWmU2fL7W2GOcZHU.jpg
10075,Roma,2018,WRITING (Original Screenplay),Written by Alfonso Cuarón,False,[Drama],7.1,Cleo (Yalitza Aparicio) es la joven sirvienta ...,/dtIIyQyALk57ko5bjac7hi01YQ.jpg
10076,Vice,2018,WRITING (Original Screenplay),Written by Adam McKay,False,"[Comedy, Drama, History]",6.9,Explora la historia real sobre cómo Dick Chene...,/seqLrPNbWJdcX6PeEhBNkmxjz6h.jpg


,film,year_film,category,name,win,Genres,Rate,Description,Poster
